In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img,img_to_array
import os
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization,Input,concatenate
import cv2
import random


In [ ]:
path=('../input/dhonivskohli/')
path1=('../input/dhoni-vs-kohli-test')
test=os.listdir(path1)
df=os.listdir(path)
print(np.array(df).shape)
X_test_origin=cv2.imread('../input/dhoni-vs-kohli-test/images.jpg')
y_test_origin=[]
y_test_origin.append('0')
X_test_origin=cv2.resize(X_test_origin,(250,250))
X_test_origin=cv2.cvtColor(X_test_origin,cv2.COLOR_BGR2GRAY)
X_test_origin=np.reshape(X_test_origin,(1,62500))/255
y_test_origin=[]
y_test_origin.append('0')
y_train_origin=[]
X_train_origin=[]
for i in df :
    j=i.split('.')[0]
    if j=='dhoni' :
        y_train_origin.append('0')
    else :
        y_train_origin.append('1')
    image=cv2.imread(path+i)
    image=cv2.resize(image,(250,250))
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #image=load_img(path+i,target_size=(250,250))
    
    #image=img_to_array(image)
    X_train_origin.append(image)
        

X_train=np.reshape(X_train_origin,(14,62500))/255
print(np.array(X_test_origin).shape)
print(y_train_origin)

In [ ]:
def plot_triplets(triplets) :
    plt.figure(figsize=(9,3))
    for i in range(0,3) :
        plt.subplot(1,3,i+1)
        plt.imshow(np.reshape(triplets[i],(250,250)),cmap='binary')
        
    plt.show()
    
plot_triplets([X_train_origin[0],X_train_origin[1],X_train_origin[2]])

In [ ]:
plot_triplets([X_train_origin[0],X_train_origin[1],X_train_origin[2]])

In [ ]:
y_train=np.array(y_train_origin)
print(np.where(y_train=='1'))

In [ ]:
def create_batch(batch_size) :
    anchors=np.zeros((batch_size,62500))
    positives=gray =np.zeros((batch_size,62500))
    negitives=np.zeros((batch_size,62500))
    for i in range(0,batch_size) :
        index=random.randint(0,13)
        anc=X_train[index]
        y=y_train[index]
        print(y)
        indices_for_pos=np.squeeze(np.where(y_train==y))
        indices_for_neg=np.squeeze(np.where(y_train!=y))
        print(indices_for_pos)
        pos=X_train[indices_for_pos[random.randint(0,len(indices_for_pos)-1)]]
        neg=X_train[indices_for_neg[random.randint(0,len(indices_for_neg)-1)]]
        anchors[i]=anc
        positives[i]=pos
        negitives[i]=neg
    return [anchors,positives,negitives]


In [ ]:
triplets=create_batch(1)

In [ ]:
plot_triplets(triplets)

In [ ]:
emb_size=128
from keras.models import Sequential,Model
from keras.layers import Dense
model=Sequential()
model.add(Dense(128,activation='relu',input_shape=(62500,)))
model.add(Dense(emb_size,activation='sigmoid'))
model.summary()
    


In [ ]:
example=X_train[0]
example_emb=model(np.expand_dims(example,axis=0))
example_emb

In [ ]:
in_anc=Input(shape=(62500,))
in_pos=Input(shape=(62500,))
in_neg=Input(shape=(62500,))
emb_anc=model(in_anc)
emb_pos=model(in_pos)
emb_neg=model(in_neg)
out=concatenate([emb_anc,emb_pos,emb_neg],axis=1)
Net=Model([in_anc,in_pos,in_neg],out)
Net.summary()

In [ ]:
def triple_loss(alpha,emb_size) :
    def loss(y_true,y_pred) :
        anc,pos,neg=y_pred[:,:emb_size],y_pred[:,emb_size:2*emb_size],y_pred[:,2*emb_size:]
        dp=tf.reduce_mean(tf.square(anc-pos),axis=1)
        dn=tf.reduce_mean(tf.square(anc-neg),axis=1)
        L=tf.max(dp-dn+alpha,0)
        return L

    return loss

In [ ]:

def data_generator(batch_size,emb_size) :
    while True :
        x=create_batch(batch_size)
        y=np.zeros(batch_size,3*emb_size)
        yield x,y
    
        



In [ ]:
epochs=10
batch_size=3
steps_per_epoch=int(14/batch_size)
print(steps_per_epoch)

In [ ]:
epochs=10
batch_size=3

steps_per_epoch=int(14/batch_size)
Net.compile(loss=triple_loss(alpha=0.2,emb_size=emb_size),optimizer='adam')
Net.fit(data_generator(batch_size,emb_size),
       epochs=epochs,
        steps_per_epoch=steps_per_epoch
       )
    
    
    


Net.predict(X_test_origin)